In [1]:
import pandas as pd
import numpy as np
import datetime
from pathlib import Path

#### Set Display

In [2]:
pd.set_option('display.max.columns', 100)
pd.set_option('display.max.rows', 1000)

#### 파일 불러오기

In [3]:
project_folder_name = '직원정보작업폴더'
project_folder_path = Path.home().joinpath('Desktop', project_folder_name).absolute()
working_folder_path = project_folder_path.joinpath('202109151350')

df = pd.read_csv(working_folder_path / '사원명부.csv', encoding='utf-8')

#### 필요한 컬럼명 정의

In [4]:
arrColumns=['사원','사번', '주민등록번호M', '성별', '나이' ,'부서','직급', '직위', '직책', '사원구분', '입사일','현부서근무일', '현직급근무일', '현직책근무일',
            '2015년 종합근무성적평가 결과', '2016년 종합근무성적평가 결과', '2017년 종합근무성적평가 결과','2018년 종합근무성적평가 결과','2019년 종합근무성적평가 결과','2020년 종합근무성적평가 결과']

#### 필요한 컬럼만 필터<br><br>아무런 정보도 없는 행 삭제

In [5]:
df = df[arrColumns]
df = df.loc[df['사원'].dropna().index]

#### 발령내역 csv 파일 불러온 후 필요한 컬럼만 필터

In [6]:
move_df = pd.read_csv(working_folder_path / '발령내역.csv', encoding='utf-8')
col=['사원','사번','발령일','소속부서','직급','발령명', '비고']
move_df = move_df[col]
move_df

,사원,사번,발령일,소속부서,직급,발령명,비고
0,한창훈,200411001.0,2021-01-11,기술보호지원부,1급,보직변경,직제개편
1,한창훈,200411001.0,2019-05-21,기술보호지원부,1급,승급,승급
2,한창훈,200411001.0,2018-04-02,기술보호지원부,2급,직무대행해제,직무대행 해제
3,한창훈,200411001.0,2018-03-02,기술보호지원부,2급,직무대행,직무대행
4,한창훈,200411001.0,2017-03-06,기술보호지원부,2급,보직변경,NaN
...,...,...,...,...,...,...,...
2109,이의선,202107007.0,2021-07-28,상생협력지원부,전문사무직 다급,입사(일반),NaN
2110,김슬빛나,202108001.0,2021-08-04,상생조정지원부,5급,입사(일반),NaN
2111,임치시스템관리,202108002.0,2021-08-09,기술임치운영부,6급,입사(일반),NaN
2112,이영숙,202109001.0,2021-09-06,상생협력본부,본부장,입사(일반),NaN


#### Function : 발령이력 정리(인자는 df) 

In [7]:
def create_history(temp) :
    # df를 역순으로 재배치
    temp = temp[::-1]
    # df의 인덱스 추출
    temp_index = temp.index

    elements = []
    for k in temp_index :
        # 각 발령 정보 추출
        date = temp.loc[k, '발령일']
        depart = temp.loc[k, '소속부서']
        grade = temp.loc[k, '직급']
        order_name = temp.loc[k, '발령명']
        etc = temp.loc[k, '비고']
        # 비고란에 내용 없으면 na값 대신 공백 처리
        if pd.isna(etc):
            etc = ""
        # 한줄 메세지 생성하고 리스트에 저장
        element = '{0}  {1} ({2})   {3}'.format(date, depart, grade, etc)
        elements.append(element)
    # 리스트에 모여있는 각 한줄 메세지들을 하나의 단락으로 병합
    final = "\n".join(elements)
    return final

#### 사원명부에 있는 사원 별로 발령이력을 하나씩 생성

In [8]:
historys = []
for k in range(0, len(df)) :
    name = df.loc[k, '사원']
    temp = move_df[move_df['사원']==name]
    history = create_history(temp)
    historys.append(history)
    
# 마지막 결과물을 df에 발령일력 컬럼 생성
df['주요 발령사항'] = historys
# df

#### 각 연도별 근무성적 컬럼이 비어 있거나 '0'으로 표기 되어 있으면 '-'표시로 변경

In [9]:
# 근무성적과 관련된 컬럼 정의
col_ = ['2015년 종합근무성적평가 결과', '2016년 종합근무성적평가 결과', '2017년 종합근무성적평가 결과','2018년 종합근무성적평가 결과','2019년 종합근무성적평가 결과','2020년 종합근무성적평가 결과']

for c in col_ :    
    df[c].fillna("-", inplace=True)
    df[c].replace({np.nan: '-', '0': '-'}, inplace=True)

# 각 연도별 근무성적 컬럼을 하나의 컬럼으로 병합(각 결과 사이는 줄바꿈)
df['근무성적'] = '2015년 '+df['2015년 종합근무성적평가 결과']+"\n"+'2016년 '+df['2016년 종합근무성적평가 결과']+"\n"+'2017년 '+df['2017년 종합근무성적평가 결과']+"\n"+'2018년 '+df['2018년 종합근무성적평가 결과']+"\n"+'2019년 '+df['2019년 종합근무성적평가 결과']+"\n"+'2020년 '+df['2020년 종합근무성적평가 결과']

df = df.drop(col_, axis=1)

#### 외부경력 파일 불러오고 필요한 컬럼만 필터 <br> (주의사항) 외부경력에 특수기호 등이 있을 경우 나중에 csv로 저장하는 과정에서 인코딩 에러가 발생하기 때문에 전처리 필요 

In [10]:
career_df = pd.read_csv(working_folder_path / '외부경력.csv', encoding='utf-8')
col=['사원','사번','직장명','입사일','퇴사일','기타담당업무']
career_df = career_df[col]

In [11]:
con_1 = career_df['직장명'].str.extract(r'([^가-힣a-zA-Z()\s]+)').notnull().squeeze()
# con_1 = career_df['직장명'].str.extract(r'^([(]+)$').notnull().squeeze()  
career_df.loc[con_1]

,사원,사번,직장명,입사일,퇴사일,기타담당업무
30,신용준,201008001,"세경대학교, 상지영서대학교 등 6개 대학 및 대학원",2002-03-01,2004-02-28,경영 및 무역관련 강의
37,김호영,201006001,㈜선경홀로그램,2005-01-14,2010-05-31,신규연구 및 정부과제진행
39,최연자,201105004,하다루㈜,2008-03-01,2009-04-30,관리총괄
40,최연자,201105004,㈜테크잇,2005-11-01,2007-05-31,관리총괄
41,최연자,201105004,㈜부경디지텍,2002-07-01,2005-08-30,"회계, 재무, 총무"
64,김영수,201408002,㈜루민비젼,2011-11-14,2012-11-17,SI 사업제안 및 구축
65,김영수,201408002,㈜누리플랜,2009-05-06,2011-10-31,u-City 사업제안 및ㅁ 구축
71,유지연,201408006,서울신문사-연세대학교,2013-03-01,2013-12-31,NaN
80,이헌,200902002,㈜한국소프트창업자문,2000-01-01,2000-12-31,"컨설팅, 지역활성화"
141,김경무,201105001,㈜대대푸드원,2007-04-16,2011-05-18,재무담당임원


#### 외부경력 정리(인자는 df)

In [12]:
def create_career(temp) :
    # df를 역순으로 재배치
    temp = temp[::-1]
    # df의 인덱스 추출
    temp_index = temp.index

    elements = []
    for k in temp_index :
        # 각 외부경력 정보 추출
        s_date = temp.loc[k, '입사일']
        e_date = temp.loc[k, '퇴사일']
        firm = temp.loc[k, '직장명']
        detail = temp.loc[k, '기타담당업무']
        # 비고란에 내용 없으면 na값 대신 공백 처리
        if pd.isna(detail):
            detail = ""
        # 한줄 메세지 생성하고 리스트에 저장
        element = '{0} ~ {1}  ({2})  {3}'.format(s_date, e_date, firm, detail)
        elements.append(element)
    # 리스트에 모여있는 각 한줄 메세지들을 하나의 단락으로 병합
    final = "\n".join(elements)
    return final

#### 직장명이 없는 경력은 삭제

In [13]:
career_df=career_df.loc[career_df['직장명'].dropna().index]

In [14]:
con = career_df['직장명'].isnull()
career_df.loc[con]

,사원,사번,직장명,입사일,퇴사일,기타담당업무


#### 사원명부에 있는 사원 별로 외부경력 이력을 하나씩 생성

In [15]:
careers = [] 
for k in range(0, len(df)) :
    name = df.loc[k, '사원']
    temp = career_df[career_df['사원']==name]
    career = create_career(temp)
    careers.append(career)
# df에 외부경력 이력 삽입
df['주요 외부경력'] = careers

#### 학사정보 csv 파일 불러온 후 필요한 컬럼만 필터 <br> (주의) 학사정보 파일은 encoding='CP949'로 불러와야함

In [16]:
accademic_df = pd.read_csv(working_folder_path / '학사정보.csv', encoding='utf-8')
col=['사원','사번','학교', '기타학교명','전공학과','입학년월','졸업년월', '학위구분', '졸업구분']
accademic_df = accademic_df[col]

#### 학사 순서 설정

In [17]:
accademic_df=accademic_df.assign(finalState=accademic_df.apply(lambda x: accademic_df.loc[x.name, '졸업구분'] if pd.isna(accademic_df.loc[x.name, '학위구분']) else accademic_df.loc[x.name, '학위구분'], axis=1))
unique = accademic_df['finalState'].unique().tolist()
print(unique)

['학사', '석사', '대졸', '전문학사', '박사', '편입', '박사수료', '석사수료', '고졸', '대학원졸업(석사)', '대학교', '대학원졸업(박사)', '대퇴', '중퇴', '전문대졸', '대학원자퇴(석사과정)']


In [18]:
rank = [1, 10, 2, .9, 11, .95, 10.9, 9.9, 0, 10, 1, 11, 15,15, .9, 9.8]
orderDic = {f'{unique[k]}' : rank[k] for k in range(0, len(unique))}
print(orderDic)

{'학사': 1, '석사': 10, '대졸': 2, '전문학사': 0.9, '박사': 11, '편입': 0.95, '박사수료': 10.9, '석사수료': 9.9, '고졸': 0, '대학원졸업(석사)': 10, '대학교': 1, '대학원졸업(박사)': 11, '대퇴': 15, '중퇴': 15, '전문대졸': 0.9, '대학원자퇴(석사과정)': 9.8}


In [19]:
accademic_df=accademic_df.assign(order=accademic_df['finalState'].apply(lambda x: orderDic[x]))
accademic_df

,사원,사번,학교,기타학교명,전공학과,입학년월,졸업년월,학위구분,졸업구분,finalState,order
0,한창훈,200411001,건국대학교,사범대학,일어교육학(학사),Mar-93,Feb-99,학사,대졸,학사,1.00
1,한창훈,200411001,고려대학교,일반대학원,일본어과,Mar-99,Feb-00,석사,대학원수료(석사과정),석사,10.00
2,정선용,200501001,전남대학교,경영대학,경제학(학사),Mar-77,Aug-84,학사,대졸,학사,1.00
3,정선용,200501001,서강대학교,언론대학원 부설 방송아카데미,방송경영관리자과정(수료),Jan-95,Jun-95,NaN,대졸,대졸,2.00
4,정재인,200501002,가톨릭대학교,전자공학부,정보통신공학(학사),Mar-96,Feb-03,학사,대졸,학사,1.00
5,제갈현실,200501003,이화여자대학교,인문과학대학,중어중문학(학사),Mar-96,Feb-00,학사,대졸,학사,1.00
6,추지은,200501004,군산서해대학,NaN,관광과(전문학사),Mar-98,Feb-00,전문학사,전문대졸,전문학사,0.90
7,형준호,200501005,서울시립대학교,NaN,무역학(학사),Mar-92,Feb-99,학사,대졸,학사,1.00
8,형준호,200501005,서울시립대학교,NaN,경제학(석사),Mar-99,Feb-02,석사,대학원졸업(석사),석사,10.00
9,조태용,200508001,원광대학교,공과대학,전자공학(학사),NaN,Feb-87,학사,대졸,학사,1.00


In [20]:
# 학사 순서대로 재배열
# create_accademic 안에서 인덱스 순서대로 for loop를 돌기 때문에 index를 reset함
accademic_df=accademic_df.sort_values(by='order')
accademic_df=accademic_df.reset_index(drop=True)

#### 학사정보 정리(인자는 df)

In [21]:
def create_accademic(temp) :
    # df를 역순으로 재배치
    # temp = temp[::-1]
    # df의 인덱스 추출
    temp_index = temp.index

    elements = []
    for k in temp_index :
        # 각 학사정보 추출
        kind = temp.loc[k, '학위구분']
        if pd.isna(kind):
            kind = temp.loc[k, '졸업구분']
        accademy = temp.loc[k, '학교']
        if pd.isna(accademy):
            accademy = temp.loc[k, '기타학교명']
        major = temp.loc[k, '전공학과']
        # 전공학과 내용 없으면 na값 대신 공백 처리
        if pd.isna(major):
            major = ""
        # 한줄 메세지 생성하고 리스트에 저장
        element = '({0})  {1}  {2}'.format(kind, accademy, major)
        elements.append(element)
    # 리스트에 모여있는 각 한줄 메세지들을 하나의 단락으로 병합
    final = "\n".join(elements)
    return final

#### 사원명부에 있는 사원 별로 학사정보 하나씩 생성

In [22]:
accademics = [] 
for k in range(0, len(df)) :
    name = df.loc[k, '사원']
    temp = accademic_df[accademic_df['사원']==name]
    accademic = create_accademic(temp)
    accademics.append(accademic)
# df에 외부경력 이력 삽입
df['학사 정보'] = accademics

In [23]:
def create_age(df) :
    df['주민등록번호M'] = df['주민등록번호M'].apply(lambda x : '({0}.{1}.{2})'.format(x[0:2], x[2:4], x[4:6]))
    df['나이'] = df['나이'].apply(lambda x: int(x)).astype('str') + '\n' + df['주민등록번호M']
    return df

In [24]:
def load_file_utf8(working_folder_path, file_name) :
    file = pd.read_csv('{0}/{1}.csv'.format(working_folder_path, file_name), encoding='utf-8')
    return file

In [25]:
def delet(df, working_folder_path):
    rubish = load_file_utf8(working_folder_path, '지우기목록')
    indice=[]
    for rubish in rubish['사원'] :
        index=df[df['사원']==rubish].index[0]
        indice.append(index)
    df=df.drop(indice)
    return df

In [26]:
def tem_employee(df, working_folder_path):
    tem = load_file_utf8(working_folder_path, '계약직목록')
    for t in tem['사원'] :
        df.loc[(df['사원'] == t), '직급'] = '5급(계약직)'
        df.loc[(df['사원'] == t), '사원구분'] = '계약직'
    return df

In [27]:
def arrange_position(df, working_folder_path) :
    order_df = load_file_utf8(working_folder_path, '직급순서')
    df_merge = pd.merge(order_df, df, left_on='직급', right_on='직급')
    df_merge = df_merge.sort_values(by=['직급순서'])
    return df_merge

In [28]:
def arrange_department(df, date, working_folder_path) :
    order_df = load_file_utf8(working_folder_path, '직제순서')
    order_df['시점']=pd.to_datetime(order_df['시점'], format='%Y-%m-%d')
    order_df=order_df[order_df['시점']==date]
    df_merge = pd.merge(order_df, df, left_on='부서', right_on='부서')
    df_merge = df_merge.sort_values(by=['직제순서'])
    return df_merge

In [29]:
df = create_age(df)

In [30]:
df = delet(df, working_folder_path)

In [31]:
df = tem_employee(df, working_folder_path)

In [32]:
cond = df.loc[:, '사원']=='구윤미'
df.loc[cond]

,사원,사번,주민등록번호M,성별,나이,부서,직급,직위,직책,사원구분,입사일,현부서근무일,현직급근무일,현직책근무일,주요 발령사항,근무성적,주요 외부경력,학사 정보
109,구윤미,202104005,(96.10.30),여자,26\n(96.10.30),기술보호지원부,5급(계약직),사원,NaN,계약직,2021-04-05,2021-04-05,2021-04-05,2021-04-05,2021-04-05 기술보호지원부 (5급),2015년 -\n2016년 -\n2017년 -\n2018년 -\n2019년 -\n2...,2019-08-05 ~ 2019-12-31 (국회) \n2020-01-01 ~ ...,(편입) 홍익대학교 글로벌경영학과\n(대퇴) 서울신학대 영어과


In [33]:
df = arrange_position(df, working_folder_path)

In [34]:
date = datetime.datetime(2021, 1, 1)
df = arrange_department(df, date, working_folder_path)

In [35]:
df = df.sort_values(by=['직제순서', '직급순서', '현직급근무일', '입사일', '사원'])
df = df.drop(['직제순서', '시점', '직급순서', '주민등록번호M', '사번'], axis=1)

In [36]:
df = df.replace({'일반직' : '정규직'})

In [37]:
# df 컬럼 순서 재배열
df=df[['사원', '성별', '나이', '부서', '직급', '직위', '직책', '사원구분',
       '입사일', '현부서근무일', '현직급근무일', '현직책근무일', '주요 발령사항', '근무성적', '학사 정보', '주요 외부경력']]
# df저장
timestamp = datetime.datetime.today().strftime('%Y%m%d_%H%M%S')
# df.to_csv(working_folder_path / '직원종합정보.csv')
df.to_csv("{0:s}/{1}직원종합정보.csv".format(str(working_folder_path), timestamp))
df.to_csv("{0:s}/{1}직원종합정보.csv".format(str(working_folder_path), timestamp), encoding='CP949')